# Лабораторная работа 6. Вариант 1. Группа 343

## Задание

Написать программу, решающую задачу указанным проекционным методом и методом коллокации.



## Условие

1) Граничная задача
$-(\frac{1}{2+x}u')'+cos(x)u=1+x, u(-1)=u(1)=0$

2) Метод Ритца

   Координатная система: $(1-x^2)P_i^{(1,1)}(x),i=0,1,...$
   
3) Метод коллокации

In [2]:
import numpy as np
import scipy.integrate
from numpy import linalg as LA
from tabulate import tabulate

N_RANGE = (3, 7)


def find_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    if b.size != n:
        raise ValueError('Incompatible sizes of A and b')

    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if A[pivot_index, i] == 0:
                raise ValueError('A is singular')

            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
        else:
            if A[i, i] == 0:
                raise ValueError('Pivot is zero')

        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x


def integrate(f, a, b):
    return scipy.integrate.trapz(f, a, b)[0]


def p(x):
    return 1 / (2 + x)


def r(x):
    return np.cos(x)


def f(x):
    return 1


def p(i, x)
    if i == 0:
        return 1
    if i == 1:
        return 2 * x
    # k = 1
    # n + 2 -> i
    # n = i - 2
    num = (i + 1) * (2 * i + 1) * x * p(i - 1, x)
    num -= (i + 1) * i * p(i - 2, x)
    denum = (i + 2) * i
    return num / denum

PS = []
for i in range(0, N_RANGE[1]):
    PS.append(p(i))
    
def w_i(i, x):
    return (1 - x ** 2) * PS[i](x)

    
def dw_i(i, x):
    return 0
    

def d2w_i(i, x):
    return 0

In [ ]:
def find_cond(A):
    norm_a = LA.norm(A, np.inf)
    inverse_a = LA.inv(A)
    norm_inverse_a = LA.norm(inverse_a, np.inf)
    return norm_a * norm_inverse_a


def find_solution_ritz(x, n):
    def find_a():
        return np.zeros((n, n))
    
    A = find_a()
    
    def find_fs():
        return 0
                 
    fs = find_fs()
    
    def find_cs():
        return find_solution_gauss(A, fs)
    
    cs = find_cs()
    
    res = 0
    for i in range(0, n):
        res += cs[i] * w_i(i, x)

In [ ]:
def find_solution_collocation():
    pass